In [ ]:
#imports
import requests
import pandas as pd
import os
from datetime import datetime
from functools import reduce

# Column names for CSV files after downloading since it doesnt have any labels
column_names = [
    'cty', 'region', 'state', 'year', 'month', 'prcp',
    '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
    '11', '12', '13', '14', '15', '16', '17', '18', '19',
    '20', '21', '22', '23', '24', '25', '26', '27', '28',
    '29', '30', '31'
]

def download_and_process_data(year, month, data_type, scale, base_directory):
    base_url = "https://www.ncei.noaa.gov/pub/data/daily-grids/v1-0-0/averages/"
    target_file = f"{data_type}-{year}{month:02d}-{scale}.csv"
    file_url = f"{base_url}{year}/{target_file}"
    
    try:
        response = requests.get(file_url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error downloading {file_url}: {e}")
        return None

    temp_directory = os.path.join(base_directory, "temp")
    if not os.path.exists(temp_directory):
        os.makedirs(temp_directory)
    temp_file_path = os.path.join(temp_directory, target_file)
    with open(temp_file_path, 'w') as file:
        file.write(response.text)
    
    return process_file(temp_file_path, data_type, year, month, scale, base_directory)

def process_file(file_path, data_type, year, month, scale, base_directory):
    df = pd.read_csv(file_path, names=column_names, header=None)

    filtered_df = df[df['state'].str.startswith('VA:')]
    new_data = []
    for index, row in filtered_df.iterrows():
        for day in range(1, 32):
            day_str = str(day)
            if day_str in filtered_df.columns:
                value = row[day_str]
                new_row = {
                    'date': f"{row['month']}/{day}/{row['year']}",
                    'year': row['year'],
                    'month': row['month'],
                    'day': day,
                    'state': row['state'][:2],
                    'county': row['state'][3:],
                    'region': row['region'],
                    'value': value,
                    'data_type': data_type
                }
                new_data.append(new_row)

    new_df = pd.DataFrame(new_data)
    
    transformed_directory = os.path.join(base_directory, f"{year}", f"{scale}_transformed")
    if not os.path.exists(transformed_directory):
        os.makedirs(transformed_directory)
    transformed_file_name = f"{year}{str(month).zfill(2)}_{data_type}_transformed.csv"
    transformed_file_path = os.path.join(transformed_directory, transformed_file_name)
    new_df.to_csv(transformed_file_path, index=False)
    
    print(f"Processed and saved transformed data: {transformed_file_path}")
    return transformed_file_path

def merge_processed_files(processed_files, base_output_path, year, scale):
    dfs = []
    for file_path in processed_files:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            dfs.append(df)
    
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        pivoted_df = combined_df.pivot_table(index=['date', 'year', 'month', 'day', 'state', 'county', 'region'],
                                             columns='data_type', values='value', aggfunc='first').reset_index()
        
        
        pivoted_df.columns.name = None
        pivoted_df = pivoted_df.rename(columns={col: f"{col}" for col in pivoted_df.columns})

        final_csv_file_path = os.path.join(base_output_path, f"{year}_{scale}_merged.csv")
        pivoted_df.to_csv(final_csv_file_path, index=False)
        print(f"All files for {year} and scale {scale} have been successfully merged into {final_csv_file_path}.")

def process_files_for_year_and_scale(year, data_types, scale, base_directory):
    processed_files = []
    for month in range(1, 13):
        for data_type in data_types:
            file_path = download_and_process_data(year, month, data_type, scale, base_directory)
            if file_path:
                processed_files.append(file_path)
    return processed_files

def process_range(start_date, end_date, data_types, scale, base_directory):
    # Initialize an empty list outside the loop to collect processed file paths for each year
    processed_files_for_year = []
    current_year = start_date.year  # Track the current year being processed

    for current_date in pd.date_range(start=start_date, end=end_date, freq='MS'):
        year = current_date.year
        month = current_date.month

        
        if year != current_year:
            
            if processed_files_for_year:
                base_output_path = os.path.join(base_directory, f"{current_year}", f"{scale}_merged")
                if not os.path.exists(base_output_path):
                    os.makedirs(base_output_path)
                merge_processed_files(processed_files_for_year, base_output_path, current_year, scale)
                processed_files_for_year = []  
            
            current_year = year  

        
        for data_type in data_types:
            file_path = download_and_process_data(year, month, data_type, scale, base_directory)
            if file_path:
                processed_files_for_year.append(file_path)
    
    
    if processed_files_for_year:
        base_output_path = os.path.join(base_directory, f"{year}", f"{scale}_merged")
        if not os.path.exists(base_output_path):
            os.makedirs(base_output_path)
        merge_processed_files(processed_files_for_year, base_output_path, year, scale)



# Give start date and end date
#give your local path
start_date = datetime(1990, 1, 1)
end_date = datetime(2023, 12, 31)
data_types = ["prcp", "tavg", "tmin", "tmax"]
scale = 'cty-scaled'
base_directory = "/Users/vaidhikraj/Downloads/"


process_range(start_date, end_date, data_types, scale, base_directory)

In [4]:


import os
import pandas as pd

directory = '/Users/vaidhikraj/Downloads/virginia_1990-2023/'  # Update with the directory containing your CSV files


dataframes = []


for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)


merged_df = pd.concat(dataframes, ignore_index=True)


output_csv_file_path = '/Users/vaidhikraj/Downloads/virginia_1990-2023/merged/virginia_1990-2023.csv'


merged_df.to_csv(output_csv_file_path, index=False)

print(f"Merged data saved to: {output_csv_file_path}")

Merged data saved to: /Users/vaidhikraj/Downloads/virginia_1990-2023/merged/virginia_1990-2023.csv


,date,year,month,day,state,county,region,prcp,tavg,tmax,tmin
1129392,1/1/2023,2023,1,1,VA,Accomack County,44001,17.98,12.77,16.28,9.27
1129393,1/1/2023,2023,1,1,VA,Albemarle County,44003,7.88,10.27,14.42,6.12
1129394,1/1/2023,2023,1,1,VA,Alexandria city,44510,6.10,10.31,14.08,6.55
1129395,1/1/2023,2023,1,1,VA,Alleghany County,44005,6.61,7.61,10.98,4.25
1129396,1/1/2023,2023,1,1,VA,Amelia County,44007,14.86,10.72,15.89,5.56
...,...,...,...,...,...,...,...,...,...,...,...
1178491,9/9/2023,2023,9,9,VA,Williamsburg city,44830,0.00,27.36,33.10,21.62
1178492,9/9/2023,2023,9,9,VA,Winchester city,44840,12.62,23.66,29.83,17.49
1178493,9/9/2023,2023,9,9,VA,Wise County,44195,2.41,21.47,25.79,17.16
1178494,9/9/2023,2023,9,9,VA,Wythe County,44197,4.49,21.23,27.36,15.10


In [4]:
#redudancy removal

import pandas as pd
import os


file_path = "/Users/vaidhikraj/Downloads/virginia_1990-2023/merged/virginia_1990-2023.csv"


df = pd.read_csv(file_path)


date_table = df[['year', 'month', 'day']].drop_duplicates().reset_index(drop=True)
date_table['DateID'] = date_table.index + 1


county_table = df[['state', 'county', 'region']].drop_duplicates().reset_index(drop=True)
county_table['CountyID'] = county_table.index + 1


weather_data = df.merge(date_table, on=['year', 'month', 'day'])
weather_data = weather_data.merge(county_table, on=['state', 'county', 'region'])
weather_data_table = weather_data[['DateID', 'CountyID', 'prcp', 'tavg', 'tmin', 'tmax']]
weather_data_table['WeatherDataID'] = weather_data_table.index + 1


date_table_file = os.path.join(os.path.dirname(file_path), 'date_table.csv')
county_table_file = os.path.join(os.path.dirname(file_path), 'county_table.csv')
weather_data_table_file = os.path.join(os.path.dirname(file_path), 'weather_data_table.csv')


date_table.to_csv(date_table_file, index=False)
county_table.to_csv(county_table_file, index=False)
weather_data_table.to_csv(weather_data_table_file, index=False)

print("Tables saved successfully.")


/var/folders/47/w_v05zrj7n1b0dm26jvljqyh0000gn/T/ipykernel_48397/698381393.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data_table['WeatherDataID'] = weather_data_table.index + 1


Tables saved successfully.
